In [1]:
from IPython.core.display import HTML
import logging, json
import cobra

In [2]:
import biosapi

In [31]:

%run ../../scripts/bios_utils.py


In [2]:
model_id = 'iMM904'

In [3]:

api = biosapi.BIOS()

In [95]:
request_url = '{}/api/neo4j/data/{}'.format(api.base_url, 3780473)
resp = requests.get(request_url, headers=api.headers)
HTML(resp.content.decode('utf-8'))

In [115]:
for rxn in model_rxns:
    sbo = get_sbo_terms(rxn['bios_id'])
    name = None
    if 'name' in rxn:
        name = rxn['name']
    print(sbo, rxn['bios_id'], name, rxn['id'])
    break

None 3154577 Glycoaldehydye reversible transport R_GCALDt


In [109]:
def get_sbo_terms(node_id):
    data = api.get_bios_node_edges(node_id)
    sboterms = get_sbo_term(data)
    terms = set()
    for id in sboterms:
        sboterm_id = sboterms[id]['bios_other_node_id']
        #print(id, edge_pros)
        sboterm = api.get_bios_node(sboterm_id)
        print(sboterm)
        terms.add(sboterm['entry'])

In [111]:
#make this generic to any edge type
def get_sbo_term(edge_data):
    if 'OUTGOING' in edge_data and 'has_sbo_term' in edge_data['OUTGOING']:
        return edge_data['OUTGOING']['has_sbo_term']
    return {}

In [ ]:
request_url = '{}/api/model/{}'.format(api.base_url, model_id)
resp = requests.get(request_url, headers=api.headers)

In [13]:
request_url = '{}/api/model/{}'.format(api.base_url, model_id)
resp = requests.get(request_url, headers=api.headers)

In [35]:

model_cmps = api.get_model_compartments(model_id)
model_spis = api.get_model_species(model_id)
model_rxns = api.get_model_reactions(model_id)
model_genes = api.get_model_genes(model_id)

In [36]:
%run ../../scripts/bios_metabolic_model.py
cobra_model = BiosModelToCobraBuilder(model_cmps, model_spis, model_rxns, model_gprs).build()
cobra_model

{'R_BIOMASS_SC5_notrace'}


Name,model
Memory address,0x013267755f8
Number of metabolites,1226
Number of reactions,1577
Number of groups,0
Objective expression,1.0*R_BIOMASS_SC5_notrace - 1.0*R_BIOMASS_SC5_notrace_reverse_353ca
Compartments,"c, r, m, e, x, n, v, g"


In [41]:
fmodels = [
    'iNL895', #
    'iCT646', #
    'iMM904', #
    'iTO977', #
    'iSS884', #
    'iLC915', #    
    'iAL1006', #
    'iRL766', #
    'iMA871', #
    'iJDZ836', #
    'iWV1314', #
    
    'iOD907', ##
    'iWV1213',
    'iJL1454',
    'iNX804',
    'yeast_6.06',
    'yeast_7.6',
]
for m in models:
    if m['entry'] in fmodels:
        print(m['entry'])

iOD907
iNL895
iWV1213
iRL766
iMM904
iLC915
iJDZ836
iMA871
iSS884
iAL1006
iTO977
iCT646


In [8]:
request_url = '{}/api/model'.format(api.base_url, model_id)
resp = requests.get(request_url, headers=api.headers)
models = resp.json()

In [26]:
#models

In [22]:
#data = resp.json()

In [15]:
data.keys()

dict_keys(['id', 'entry', 'name', 'source', 'version', 'description', 'md5', 'reactions', 'metabolites', 'species', 'subcellularCompartments', 'subsystems', 'genes', 'properties'])

In [162]:
logger = logging.getLogger(__name__)

In [21]:
cobra_model.summary()

IN FLUXES            OUT FLUXES            OBJECTIVES
-------------------  --------------------  -------------------
M_4abz_e    10       M_asn__L_e    10      R_BIOMASS_SC...  10
M_dttp_e    10       M_cys__L_e    10
M_gam6p_e   10       M_gal_e       10
M_hxan_e    10       M_gln__L_e    10
M_iamoh_e   10       M_pi_e        10
M_tre_e     10       M_thymd_e     10
M_tyr__L_e  10       M_trp__L_e    10
M_xtsn_e    10       M_pap_e        9.8
M_ocdca_e    9.98    M_ocdcya_e     9.48
M_ptrc_e     8.17    M_acald_e      8.09
M_ins_e      7.99    M_iamac_e      7.04
M_mal__L_e   7.78    M_pyr_e        5.22
M_chol_e     6.98    M_g3pc_e       4.75
M_nmn_e      6.33    M_Nbfortyr_e   4.49
M_lac__L_e   5.12    M_ibutoh_e     4.01
M_gthox_e    5       M_nadp_e       3.33
M_2mppal_e   4.65    M_asp__L_e     3.26
M_uri_e      4.18    M_dad_2_e      3.17
M_melib_e    3.5     M_id3acald_e   2.93
M_dcyt_e     3.44    M_xylt_e       1.87
M_pro__L_e   3.25    M_dgsn_e       1.73
M_so3_e      3.02  

IN FLUXES                  OUT FLUXES             OBJECTIVES
-------------------------  ---------------------  -------------------
M_co2_e          10        M_fru_e     10         R_BIOMASS_SC...  10
M_dcyt_e         10        M_nadp_e    10
M_fum_e          10        M_thymd_e   10
M_glc__D_e       10        M_xylt_e    10
M_gua_e          10        M_cit_e      9.8
M_ins_e          10        M_pi_e       9.67
M_nmn_e          10        M_gsn_e      9.63
M_o2_e           10        M_trp__L_e   9.26
M_ptrc_e         10        M_gln__L_e   8.77
M_sbt__D_e       10        M_amet_e     7.05
M_gam6p_e         9.74     M_val__L_e   5.71
M_xtsn_e          8.68     M_alltn_e    5.31
M_inost_e         8.31     M_4abut_e    5
M_ala__L_e        7.49     M_asp__L_e   4.81
M_thr__L_e        7.47     M_ura_e      4.67
M_3c3hmp_e        6.59     M_cys__L_e   4.38
M_pap_e           6        M_iamac_e    2.98
M_gthox_e         5        M_chol_e     2.78
M_man_e           4.78     M_zymst_e    2.16
M_